In [4]:
from fastai.imports import *
from fastai.vision.all import *
from fastai.vision.widgets import *
from PIL import Image
import numpy

ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
PATH = "/kaggle/input/testset/data/validation"

In [ ]:
bs = 128

In [ ]:
im = Image.open(PATH+"/no_pools/img957.jpg")
im

In [5]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(50))


NameError: name 'DataBlock' is not defined

In [ ]:
dls = data.dataloaders(PATH, bs=bs)

In [ ]:
learn= load_learner('/kaggle/input/loadmodel5/firstModel5.pkl', cpu=False)

In [ ]:
new_learn = Learner(dls, learn.model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
new_learn.model.load_state_dict(learn.model.state_dict())


In [ ]:
val_loss, val_acc = new_learn.validate()

In [ ]:
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

In [ ]:


def generate_grad_cam(learn, image_path):
    img = PIL.Image.open(image_path)
    x, = first(learn.dls.test_dl([img]))  # Preprocess the image using the dataloader

    with hook_output(learn.model[0]) as hook_a:
        with hook_output(learn.model[0], grad=True) as hook_g:
            preds = learn.model.eval()(x.unsqueeze(0).cuda())
            preds[:, learn.pred.argmax().item()].backward()

    acts = hook_a.stored[0].cpu()
    grad = hook_g.stored[0][0].cpu()

    grad_chan = grad.mean(1).mean(1)
    mult = (acts * grad_chan[..., None, None]).mean(0)

    _, ax = plt.subplots()
    img.show(ctx=ax)
    ax.imshow(mult, alpha=0.6, extent=(0, *img.size, img.size[1], 0),
              interpolation='bilinear', cmap='inferno')
    plt.show()

# Replace 'your_image_path.jpg' with the path to the image you want to visualize
generate_grad_cam(learn, PATH+"/no_pools/img957.jpg")